In [10]:
import os
import pytesseract
from pdf2image import convert_from_path
import string 
from nltk.stem.snowball import SnowballStemmer
from googletrans import Translator
import pandas as pd

In [11]:
def File_Content(file_name):
    file_str = ''

    if file_name.endswith('.pdf'):
        pdf_file = convert_from_path(file_name)

        for i in range(len(pdf_file)):
            pdf_file[i].save(f'page{str(i + 1)}.jpg', 'JPEG')   # Saves pages as images in the pdf
            file = f'page{str(i + 1)}.jpg'
            file_str += pytesseract.image_to_string(file)
            # os.remove(file)   # Deletes saved image files
    elif file_name.endswith(('.jpg', '.png')):
        file_str = pytesseract.image_to_string(file_name)
        
    return file_str

In [12]:
file_cont= File_Content('Computer Reading.pdf')

In [13]:
with open("Deneme.txt","a",encoding="utf-8") as file:
    file.write(file_cont)  
    

In [14]:
    with open("Deneme.txt","r+",encoding="utf-8") as file:
        content= file.readlines()
        parag=  file.read()
    

        untitled=""
        for i in content:
            if(i.isupper()==True or (i.istitle()==True)):
                untitled += " "
            else:
                untitled += i + " "
    with open("Computer Reading Son.txt","a",encoding="utf-8") as file1:
        for i in untitled:
            file1.write(i)

In [15]:
import string 
from nltk.tokenize import sent_tokenize
with open("Computer Reading Son.txt","r",encoding="utf-8") as file: 
    contents=file.read()
    content_clear =contents.replace("\n","")
    content_clear = content_clear.replace("/n","")
    
    def punctuation_clear(text):
        result = ""
        for i in text:
            if( i not in string.punctuation or i == " " ):
                result+= i 
            else:
                result+= ""
        return result
    def punctuation_clear_list(text):
        result = ""
        for i in text:
            for a in i:
                if( a not in string.punctuation or a == "." or a == " " ):
                    result+= a 
            
        return result
    content_clears = punctuation_clear(content_clear)
    content_split_sentence = punctuation_clear_list(content_clear).split(".")
    content_word_split= set(content_clears.split())
    word_list= list()
    say = 0
    for i in content_word_split:
        if(len(i)>=3):
            word_list.append(i)
            

In [16]:
sentence_list_last = list()
for a in word_list:
    x=0
    for i in content_split_sentence:
        sentence = i.split()
        for j in sentence:
            if a==j:
                sentence_list_last.append(i)
                x+=1
            if x==1:
                break
        if x==1:
            break
            
content_clear_split_last = list()

423


In [17]:
from nltk.stem.snowball import SnowballStemmer
stem_list=[]

for i in word_list:
    stemmer = SnowballStemmer("english")
    stem_list.append(stemmer.stem(i))
#stem_set = set(stem_list)
#stem_list = list(stem_set)
a=  0
for i in stem_list:
    a+=1
print(a)

423


In [22]:
from googletrans import Translator
translator = Translator()
ceviri_list= list()
for i in word_list:
    translation = translator.translate(i,dest = 'tr')
    ceviri_list.append(translation.text)

In [19]:
from nltk.corpus import state_union
import nltk
from nltk.tokenize import PunktSentenceTokenizer
code_list = list()
for i in word_list:
        words = nltk.word_tokenize(i)
        tagged = nltk.pos_tag(words)
        code_list.append(tagged[0][1])
        

In [23]:
import pandas as pd
dict = {"Code:":code_list,"Raw":word_list,"Root":stem_list,"Translation":ceviri_list,"Sentence":sentence_list_last}
df= pd.DataFrame(dict)
df.to_csv("Computer Reading Translate.csv")

In [19]:
import os
import pytesseract
from pdf2image import convert_from_path
import string 
from nltk.stem.snowball import SnowballStemmer
from googletrans import Translator
import pandas as pd
import string 
from nltk.tokenize import sent_tokenize

def vocab_cam(file_name1):
    def File_Content(file_name):
        file_str = ''

        if file_name.endswith('.pdf'):
            pdf_file = convert_from_path(file_name)

            for i in range(len(pdf_file)):
                pdf_file[i].save(f'page{str(i + 1)}.jpg', 'JPEG')   # Saves pages as images in the pdf
                file = f'page{str(i + 1)}.jpg'
                file_str += pytesseract.image_to_string(file)
                # os.remove(file)   # Deletes saved image files
        elif file_name.endswith(('.jpg', '.png')):
            file_str = pytesseract.image_to_string(file_name)   
        return file_str
    file_cont= File_Content(file_name1)
    
    with open("Deneme.txt","a",encoding="utf-8") as file:
        file.write(file_cont)  
    with open("Deneme.txt","r+",encoding="utf-8") as file:
        content= file.readlines()
        parag=  file.read()
        untitled=""
        for i in content:
            if(i.isupper()==True or (i.istitle()==True)):
                untitled += " "
            else:
                untitled += i + " "
    with open("Computer Reading Son.txt","a",encoding="utf-8") as file1:
        for i in untitled:
            file1.write(i)
    with open("Computer Reading Son.txt","r",encoding="utf-8") as file:
        contents=file.read()
        content_clear =contents.replace("\n","")
        content_clear = content_clear.replace("/n","")
        def punctuation_clear(text):
            result = ""
            for i in text:
                if( i not in string.punctuation or i == " " ):
                    result+= i 
                else:
                    result+= ""
            return result                
        
        def punctuation_clear_list(text):
            result = ""
            for i in text:
                for a in i:
                    if( a not in string.punctuation or a == "." or a == " " ):
                        result+= a 
            return result
        content_clears = punctuation_clear(content_clear)
        content_split_sentence = punctuation_clear_list(content_clear).split(".")
        content_word_split= set(content_clears.split())
        word_list= list()
        
        for i in content_word_split:
            if(len(i)>=3):
                word_list.append(i)
        say1=0
        for q in word_list:
            say1+=1
        print("Kelime Listesi : ",say1)
    
    p=0
    sentence_list_last = list()
    for w in word_list:
        
        num=0
        for r in content_split_sentence:
            sentence = r.split()
            for e in sentence:
                if w==e:
                    sentence_list_last.append(r)
                    num+=1
                if num==1:
                    break
            if num==1:
                break
    say2=0
    for q in sentence_list_last:
        say2+=1
    print("Cümle Listesi : ",say2)            
    content_clear_split_last = list()
    from nltk.stem.snowball import SnowballStemmer
    stem_list=[]

    for i in word_list:
        stemmer = SnowballStemmer("english")
        stem_list.append(stemmer.stem(i))
    #stem_set = set(stem_list)
    #stem_list = list(stem_set)
    say3=0
    for q in stem_list:
        say3+=1
    print("kök Listesi : ",say3)
    from googletrans import Translator
    translator = Translator()
    ceviri_list= list()
    for i in word_list:
        translation = translator.translate(i,dest = 'tr')
        ceviri_list.append(translation.text)
    say4=0
    for q in ceviri_list:
        say4+=1
    print("çeviri Listesi : ",say4)
    from nltk.corpus import state_union
    import nltk
    from nltk.tokenize import PunktSentenceTokenizer
    code_list = list()
    for i in word_list:
        words = nltk.word_tokenize(i)
        tagged = nltk.pos_tag(words)
        code_list.append(tagged[0][1])
    say5=0
    for q in code_list:
        say5+=1
    print("kod Listesi : ",say5)
    import pandas as pd
    dict = {"Code:":code_list,"Raw":word_list,"Root":stem_list,"Translation":ceviri_list,"Sentence":sentence_list_last}
    df= pd.DataFrame(dict)
    df.to_csv("Computer Reading Translate.csv")
    return df
        




In [20]:
vocab_cam('Computer Reading.pdf')
    

Kelime Listesi :  423
Cümle Listesi :  423
kök Listesi :  423
çeviri Listesi :  423
kod Listesi :  423


,Code:,Raw,Root,Translation,Sentence
0,VBG,calculating,calcul,Hesaplanıyor,Common applications programs include wordproc...
1,RBR,more,more,daha fazla,As computer systems are developed they are be...
2,NN,copying,copi,kopyalama,Note that copying data from a larger server s...
3,NNS,graphics,graphic,grafik,A multimedia computer can process different f...
4,NN,keyboard,keyboard,tuş takımı,The main device for inputting the data is a t...
...,...,...,...,...,...
418,NN,chip’,chip,yonga',the ‘computer on a chip’
419,NNS,records,record,kayıtlar,smart cards which are able to store informati...
420,NN,security,secur,güvenlik,They can be used in a very wide variety of si...
421,VBN,searched,search,arandı,Common applications programs include wordproc...
